In [19]:
import pandas as pd
import psycopg2
import os
from sqlalchemy import create_engine
from datetime import date


# The initial data load (step 1) must be opened from a Postgres server
# The final data load (step 2) requires MySQL 

today = date.today()
directory = r"...\data\postgres\{}\{}.csv" # Choose a directory for the backup


def db_connection():  # (Step 1) Function to open the database and save the tables backup
    
    # Here you must know the credentials to connect the postegres database
    # This method is easier for running the code in different computers
    
    print('Insert host name/address')
    host = input()
    print('Insert host database name')
    dbname = input()
    print('Insert username')
    user = input()
    print('Insert password')
    password = input()

    dbconnection = "host={} dbname={} user={} password={}".format(host, dbname, user, password)
    print('Connecting into database...')
    
    try:  
        conn = psycopg2.connect(dbconnection)
        print('Connection successfull')
        
        print('Save data in local disk? Y or N')
        response = input()
        if response == "Y":
            
            csv_table = pd.read_csv("order_details.csv")
    
            cur = conn.cursor()
        
            s = ""
            s += "SELECT"
            s += " table_schema"
            s += ", table_name"
            s += " FROM information_schema.tables"
            s += " WHERE"
            s += " ("
            s += " table_schema = 'public'"
            s += " )"
            s += " ORDER BY table_schema, table_name;"
        
            cur.execute(s)
            list_tables = cur.fetchall()
        
        # Selecting each table name and saving them in local disk: 
            for t_name_table in list_tables: 
                table_name = t_name_table[1]
                df = pd.read_sql("select * from {}".format(table_name), conn)
                df.to_csv(directory.format(table_name, today))
        
            csv_table.to_csv(directory.format("order_details", today))
            
            print('Save successfull')
    
    except psycopg2.DatabaseError:
        print('Connection Error! Check login credentials')

        
directory = r"...\data\postgres\.csv" # Choose a directory to save the final query csv file

    
def final_db_load(): # (Step 2) Loading files in the final database and displaying orders and order_detail tables
    
    # Again you must know the credentials to connect in the MySQL server
    
    print("Insert username")
    user = input()
    print("Insert password")
    password = input()
    print('Insert host name/address')
    host = input()=-
    print('Insert database name')
    dbname = input()

    dbengine = "mysql+pymysql://{}:{}@{}/{}".format(user, password, host, dbname)
    engine = create_engine(dbengine)
    
    # Loading the backup files and loading into the final database:
    rootDir = r'...\data\postgres'  # Select the root file for the tables backup
    for dirName, subdirList, fileList in os.walk(rootDir, topdown=False):
        for fname in fileList:
            table_name = dirName.split('\\')[-1] 
            filepath = os.path.join(rootDir, dirName, fname)
            df = pd.read_csv(filepath)
            df.to_sql(table_name, con=engine, index = False)
        
    print("Show the orders and its details? Y or N")
    response = input()
    if response == "Y":
             
        orders = pd.read_sql("SELECT * FROM orders", engine)
        details = pd.read_sql("SELECT * FROM order_details", engine)
        
        print("Orders table:")
        display(orders)
        orders.to_csv(directory)
        print("Orders details table:")
        display(details)
        details.to_csv(directory)


In [22]:
db_connection()

Insert host name/address
127.0.0.1
Insert host database name
northwind
Insert username
postgres
Insert password
1234
Connecting into database...
Connection Successful
Save data in local disk? Y or N
Y


In [23]:
final_db_load()

Insert username
root
Insert password
1234
Insert host name/address
127.0.0.1
Insert database name
northwind2
Show the orders and its details? Y or N
Y


,Unnamed: 0,order_id,customer_id,employee_id,order_date,required_date,shipped_date,ship_via,freight,ship_name,ship_address,ship_city,ship_region,ship_postal_code,ship_country
0,0,10248,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France
1,1,10249,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany
2,2,10250,HANAR,4,1996-07-08,1996-08-05,1996-07-12,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,RJ,05454-876,Brazil
3,3,10251,VICTE,3,1996-07-08,1996-08-05,1996-07-15,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,None,69004,France
4,4,10252,SUPRD,4,1996-07-09,1996-08-06,1996-07-11,2,51.30,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,None,B-6000,Belgium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,825,11073,PERIC,2,1998-05-05,1998-06-02,None,2,24.95,Pericles Comidas clásicas,Calle Dr. Jorge Cash 321,México D.F.,None,05033,Mexico
826,826,11074,SIMOB,7,1998-05-06,1998-06-03,None,2,18.44,Simons bistro,Vinbæltet 34,Kobenhavn,None,1734,Denmark
827,827,11075,RICSU,8,1998-05-06,1998-06-03,None,2,6.19,Richter Supermarkt,Starenweg 5,Genève,None,1204,Switzerland
828,828,11076,BONAP,4,1998-05-06,1998-06-03,None,2,38.28,Bon app',"12, rue des Bouchers",Marseille,None,13008,France


,Unnamed: 0,order_id,product_id,unit_price,quantity,discount
0,0,10248,11,14.00,12,0.00
1,1,10248,42,9.80,10,0.00
2,2,10248,72,34.80,5,0.00
3,3,10249,14,18.60,9,0.00
4,4,10249,51,42.40,40,0.00
...,...,...,...,...,...,...
2150,2150,11077,64,33.25,2,0.03
2151,2151,11077,66,17.00,1,0.00
2152,2152,11077,73,15.00,2,0.01
2153,2153,11077,75,7.75,4,0.00
